# 0. Install and Import Dependencies

In [16]:
import cv2
import mediapipe as mp
import numpy as np
import math
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# 2. Determining Joints

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

# 3. Calculate Angles

In [36]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return int(angle) 

#### finding the distance 

In [30]:
def distance(x1,y1,x2,y2):
    x1 = np.array(x1)
    y1 = np.array(y1)
    x2 = np.array(x2)
    y2 = np.array(y2)
    distance = math.sqrt((x2-x1)**2 + (y2-y1)**2 )
    return int(distance)


#### finding the mid-points

In [19]:
def mid_points_x(x1,x2):
    x1 = np.array(x1)
    x2 = np.array(x2)
    mid_x = (x1 + x2) / 2

    return mid_x


In [20]:
def mid_points_y(y1,y2):
    x1 = np.array(y1)
    x2 = np.array(y2)
    mid_y = (y1 + y2) / 2

    return mid_y

#### Check Horizontal line or not


In [21]:
def find_angles(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    angle_with_x_axis = math.degrees(math.atan2(dy, dx))
    angle_with_y_axis = 90 - angle_with_x_axis
    return angle_with_x_axis, angle_with_y_axis

#### Finding the Slope 

In [22]:
def slope(x1,x2,y1,y2):
    
    dx = x2 - x1
    dy = y2 - y1

    slope = dy/dx # slope =  diff in y-axis / diff in x-axis
    slope_angle_x = math.degrees(math.atan(slope))
    slope_angle_y = 90 - slope_angle_x

    return slope_angle_x,slope_angle_y 

In [23]:
def relative(landmark, shape):
    
    x = int(landmark.x * shape[1])
    y = int(landmark.y * shape[0])

    return (x,y)

In [24]:
def set_point_at_angle(origin, angle_degrees, distance):
    # Convert the angle from degrees to radians
    angle_radians = math.radians(angle_degrees)
    
    # Calculate the X and Y coordinates of the point
    x = origin[0] + distance * math.cos(angle_radians)
    y = origin[1] + distance * math.sin(angle_radians)

    return (int(x), int(y))

# 4. Shoulder Side Detections

In [43]:
cap = cv2.VideoCapture(0)
stage = None
deflection = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = cv2.flip(image,1)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        #image = cv2.flip(image,1)
        shape =  image.shape
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            #Extracting body parts Coordinates

            # Elbow
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            
            # Shoulder
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

          #  print("shoulder keypoints --->",left_shoulder,right_shoulder)
            # Hip
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y] 
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
           # print("hip keypoints --->",left_hip,right_hip)
            # Separate x,y Coordinates

            #left_shoulder
            left_shoulder_x, left_shoulder_y = relative(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],shape)
           # print("Relative shoulder left ---->",left_shoulder_x,left_shoulder_y)
            #left_hip
            left_hip_x,left_hip_y = relative(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],shape)
          # print("Relative Hip Left--->",left_hip_x,left_hip_y)
            
            #Right_shoulder
            right_shoulder_x,right_shoulder_y = relative(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],shape)
           # print("Relative shoulder right ---->",right_shoulder_x,right_shoulder_y)
            #Right_hip
            right_hip_x,right_hip_y = relative(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],shape)
           # print("realtive hip right",right_hip_x,right_hip_y)

            # ============= Coordiantes end  =======================================================
            # Distance

            left_side = distance(left_hip_x,left_hip_y,left_shoulder_x,left_shoulder_y)
            #print("left side",left_side)
            right_side = distance(right_hip_x,right_hip_y,right_shoulder_x,right_shoulder_y)
            #print("right side",right_side)

            # Mid Points 
            mid_points_shoulder = [mid_points_x(left_shoulder_x,right_shoulder_x),mid_points_y(left_shoulder_y,right_shoulder_y)]
            #   print("mid point shoulder",mid_points_shoulder)
            mid_point_hip = [mid_points_x(left_hip_x,right_hip_x),mid_points_y(left_hip_y,right_hip_y)]

            # ============= calculate Distance =====================================================

            dis_mid_rshoulder = distance(mid_points_shoulder[0],right_shoulder_x,mid_points_shoulder[1],right_shoulder_y)
           # print("Distance of mid to right",dis_mid_rshoulder) 
            dis_mid_lshoulder = distance(mid_points_shoulder[0],left_shoulder_x,mid_points_shoulder[1],left_shoulder_y) 
           # print("Distance of mid to left",dis_mid_lshoulder) 

            # ============ Calcualte Angles ========================================================

            # Calculate angle
            left_angle = calculate_angle(left_hip,left_shoulder,right_shoulder)
            #print("left angle", left_angle)
            right_angle = calculate_angle(right_hip,right_shoulder,left_shoulder)
           # print("right angle",right_angle)
            
            # =========== Visualization==============================================================   
            # Mid Point circle
            img_h,img_w,img_c = image.shape
            
            mid_x = int(mid_points_shoulder[0])
            mid_y = int(mid_points_shoulder[1])
            mid_c = (mid_x , mid_y) # center keypoints
            
            # plot mid-points
            cv2.circle(image,mid_c,5,(0,255,0),4) #plot mid point in circle

            # ================ To Add ref-line calculate other keypoints ============================
            
            a = int(dis_mid_lshoulder) # ref var a
            b = int(dis_mid_rshoulder) # ref var b
            
            mid_left_line_end = (mid_x + a , mid_y)
            mid_right_line_end = (mid_x - b, mid_y)
            
            # Visualize ref - line ===================================================================
            cv2.line(image,mid_c,mid_left_line_end,(0,255,0),2) # from center to left
            cv2.line(image,mid_c,mid_right_line_end,(0,255,0),2) # from centerr to right

            #mid_right_angle  = calculate_angle(mid_points_shoulder,right_shoulder,right_hip)
           # mid_left_angle  = calculate_angle(mid_points_shoulder,left_shoulder,left_hip)

            # ================= MAIN LOGIC =============================================================
            
            # Posture Detections
            right_stage = (int(right_side) > int(left_side)) and (int(left_angle) > 87)
            left_stage = (int(left_side)  > int(right_side)) and (int(right_angle) > 88)

            if right_stage == True:
                stage = "Left Shoulder Up "
                ri_an =  find_angles(mid_points_shoulder[0],mid_points_shoulder[1],right_shoulder_x,right_shoulder_y)
                defl = 180  + int(ri_an[0])
                new = defl 
                deflection=str(new)
            elif left_stage == True:
                stage = "Right Shoulder Up "
                le_an = find_angles(mid_points_shoulder[0],mid_points_shoulder[1],left_shoulder_x,left_shoulder_y)
                defl = int(abs(le_an[0]))
                new = defl 
                deflection=str(new)
            elif right_stage == False and left_stage == False:
                stage = "Normal"
                le_an = find_angles(mid_points_shoulder[0],mid_points_shoulder[1],left_shoulder_x,left_shoulder_y)
                ri_an = find_angles(mid_points_shoulder[0],mid_points_shoulder[1],right_shoulder_x,right_shoulder_y)
                if ri_an[0] >= 0:
                    defl = int(abs(180 - ri_an[0])) - int(abs(le_an[0]))
                    new = defl 
                    deflection=str(new)
                else:
                    defl = int(abs(180 + ri_an[0])) - int(abs(le_an[0]))
                    new = defl 
                    deflection=str(new) 
        except:
            pass
        
        #Stage
        cv2.putText(image,'POSTURE', (15,20),
                    cv2.FONT_HERSHEY_PLAIN, 1.5, (0,0,0), 2, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (15,40), 
                    cv2.FONT_HERSHEY_DUPLEX, 0.7, (128,0,128), 2, cv2.LINE_AA)
        
        #Deflection
        cv2.putText(image,'Deviation Angle' , (15,80), 
                    cv2.FONT_HERSHEY_PLAIN, 1.5, (0,0,0), 2, cv2.LINE_AA)
        cv2.putText(image, deflection , 
                    (15,100), 
                    cv2.FONT_HERSHEY_DUPLEX, 0.7, (128,0,128), 2, cv2.LINE_AA)
        cv2.circle(image,(45,87),3,(128,0,128),2)
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 ) 
        image = cv2.resize(image,(1280,720))
        cv2.imshow('Mediapipe Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# if Code Show error and cam is open in backgroud to close cam run below cmd

In [42]:
cap.release()
cv2.destroyAllWindows()